<a href="https://colab.research.google.com/github/Arhin-Eben/Machine-learning-with-python/blob/master/Copy_of_OSVFuseNet_MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive so Colab can access files stored there
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input, callbacks
from tensorflow.keras.layers import DepthwiseConv1D, Conv1D, BatchNormalization, ReLU, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout, Flatten, concatenate, Reshape
from tensorflow.keras import backend as K
import tensorflow as tf
import re
from sklearn.metrics import accuracy_score, roc_curve, auc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# ---- DATASET EXTRACTION ----
zip_file_path = '/content/drive/MyDrive/SVC-2004_Task1.zip'
extract_dir = '/content'
if not os.path.exists('/content/Task1'):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted {zip_file_path} to {extract_dir}")
else:
    print("Dataset already extracted.")
print("Contents of /content:", os.listdir('/content'))

Dataset already extracted.
Contents of /content: ['.config', 'Task1', 'drive', 'sample_data']


In [ ]:
# --------- Feature Extraction Function ---------
def extract_handcrafted_features(sig):
    t, x, y, p = [sig[:, i] for i in range(4)]
    dt = np.diff(t) + 1e-6
    dx, dy = np.diff(x), np.diff(y)
    velocity = np.sqrt(dx**2 + dy**2) / dt
    acceleration = np.diff(velocity) / dt[1:] if len(velocity) > 1 else np.zeros(1)
    jerk = np.diff(acceleration) / dt[2:] if len(acceleration) > 1 else np.zeros(1)
    curvature = (
        np.abs(dx[1:] * dy[:-1] - dy[1:] * dx[:-1]) /
        (dx[:-1]**2 + dy[:-1]**2 + 1e-6)**1.5 if len(dx) > 1 else np.zeros(1)
    )
    # Four example features: Aspect ratio, area, baseline slant angle, pressure/velocity variance
    aspect_ratio = (np.max(x) - np.min(x)) / (np.max(y) - np.min(y) + 1e-6)
    area = (np.max(x) - np.min(x)) * (np.max(y) - np.min(y))
    baseline_slant = np.arctan2(y[-1] - y[0], x[-1] - x[0]) if len(x) > 1 else 0
    pressure_var = np.var(p)
    velocity_var = np.var(velocity)
    features = [
        t[-1] - t[0],
        np.max(velocity) if len(velocity) else 0,
        np.mean(velocity) if len(velocity) else 0,
        np.std(velocity) if len(velocity) else 0,
        np.max(acceleration) if len(acceleration) else 0,
        np.mean(acceleration) if len(acceleration) else 0,
        np.std(acceleration) if len(acceleration) else 0,
        np.max(jerk) if len(jerk) else 0,
        np.mean(jerk) if len(jerk) else 0,
        np.std(jerk) if len(jerk) else 0,
        np.max(p), np.mean(p), np.std(p),
        np.mean(curvature) if len(curvature) else 0,
        np.std(curvature) if len(curvature) else 0,
        aspect_ratio, area, baseline_slant, pressure_var, velocity_var
    ]
    return np.array(features, dtype=np.float32)

In [ ]:
# --------- Data Loading Function ---------
def load_signatures_and_labels_from_folder(folder_path):
    signatures = []
    labels = []
    user_ids = []
    print(f"Attempting to load from: {folder_path}")
    if not os.path.isdir(folder_path):
        print(f"Error: Folder not found at {folder_path}")
        return signatures, labels, user_ids
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith('.txt'):
            continue
        fpath = os.path.join(folder_path, fname)
        user_match = re.search(r'U(\d+)', fname, re.IGNORECASE)
        match = re.search(r'S(\d+)', fname, re.IGNORECASE)
        if not match or not user_match:
            print(f"Skipping {fname}: cannot extract sample/user number")
            continue
        sample_num = int(match.group(1))
        user_id = int(user_match.group(1))
        if 1 <= sample_num <= 20:
            label = 0
        elif 21 <= sample_num <= 40:
            label = 1
        else:
            print(f"Skipping {fname}: sample number out of expected range (1-40)")
            continue
        data = []
        with open(fpath, 'r') as f:
            next(f)
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 4:
                    try:
                        data.append([float(x) for x in parts[:4]])
                    except ValueError:
                        continue
        if data:
            data = np.array(data)
            if data.shape[1] == 4:
                signatures.append(data)
                labels.append(label)
                user_ids.append(user_id)
    return signatures, labels, user_ids

In [ ]:
# --------- Data Preprocessing ---------
def preprocess_signature(sig, max_len=200):
    T = sig.shape[0]
    if T < max_len:
        pad = np.zeros((max_len-T, sig.shape[1]))
        sig = np.vstack([sig, pad])
    elif T > max_len:
        sig = sig[:max_len]
    scaler = MinMaxScaler()
    sig = scaler.fit_transform(sig)
    return sig

def preprocess_dataset(signatures, max_len=200):
    X, X_hand = [], []
    for sig in signatures:
        X.append(preprocess_signature(sig, max_len))
        X_hand.append(extract_handcrafted_features(sig))
    return np.array(X, dtype=np.float32), np.array(X_hand, dtype=np.float32)

In [ ]:
# --------- Autoencoder & DWSCNN Model ---------
def build_cae_encoder(input_shape=(200,4)):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 5, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(2)(x)
    x = Conv1D(64, 3, activation='relu', padding='same')(x)
    encoded = MaxPooling1D(2)(x)
    encoder = models.Model(inputs, encoded)
    return encoder

def dws_conv_block(x, filters, kernel_size, strides=1):
    x = DepthwiseConv1D(kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def build_osvfusenet(input_shape=(200,4), num_handcrafted_features=20):
    sig_input = Input(shape=input_shape, name='signature_input')
    hand_input = Input(shape=(num_handcrafted_features,), name='handcrafted_input')
    encoder = build_cae_encoder(input_shape)
    deep_features = encoder(sig_input)
    deep_features = Flatten()(deep_features)
    fusion = concatenate([deep_features, hand_input])
    total_features = fusion.shape[-1]
    x = Reshape((total_features, 1))(fusion)
    x = dws_conv_block(x, 64, 3)
    x = MaxPooling1D(2)(x)
    x = dws_conv_block(x, 128, 3)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    return models.Model([sig_input, hand_input], output)

def compile_model(model):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# --------- 1. DWSCNN Classifier Standard Training ---------
# Hybrid Feature Fusion Train (handcrafted + deep)
dataset_dir = "/content/Task1"
signatures, labels, user_ids = load_signatures_and_labels_from_folder(dataset_dir)
MAX_LEN = 100
expected_sig_features = 4
expected_hand_features = 20  # Now using more features

X, X_hand = preprocess_dataset(signatures, max_len=MAX_LEN)
y = np.array(labels)
users = np.array(user_ids)

Attempting to load from: /content/Task1


In [ ]:
# Initial train/test split for vanilla DWSCNN (CORRECTED: split users as well)
X_train, X_test, Xh_train, Xh_test, y_train, y_test, users_train, users_test = train_test_split(
    X, X_hand, y, users, test_size=0.2, random_state=42, stratify=y
)

model = build_osvfusenet(input_shape=(MAX_LEN, expected_sig_features), num_handcrafted_features=expected_hand_features)
model = compile_model(model)
model.summary()
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    [X_train, Xh_train], y_train,
    validation_data=([X_test, Xh_test], y_test),
    epochs=20,
    batch_size=16,
    callbacks=[early_stop],
    verbose=2
)


Model: "functional_267"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ signature_input     │ (None, 100, 4)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_266      │ (None, 25, 64)    │      6,880 │ signature_input[… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_133         │ (None, 1600)      │          0 │ functional_266[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ handcrafted_input   │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_133     │ (None, 1620)      │          0 │ flatten_133[0][0… │
│ (Concatenate)       │                   │            │ handcrafted_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_133         │ (None, 1620, 1)   │          0 │ concatenate_133[… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d_2… │ (None, 1620, 1)   │          4 │ reshape_133[0][0] │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1620, 1)   │          4 │ depthwise_conv1d… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_532 (ReLU)    │ (None, 1620, 1)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_534 (Conv1D) │ (None, 1620, 64)  │        128 │ re_lu_532[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1620, 64)  │        256 │ conv1d_534[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_533 (ReLU)    │ (None, 1620, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_401   │ (None, 810, 64)   │          0 │ re_lu_533[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv1d_2… │ (None, 810, 64)   │        256 │ max_pooling1d_40… │
│ (DepthwiseConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 810, 64)   │        256 │ depthwise_conv1d… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_534 (ReLU)    │ (None, 810, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_535 (Conv1D) │ (None, 810, 128)  │      8,320 │ re_lu_534[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 810, 128)  │        512 │ conv1d_535[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 24,937 (97.41 KB)

 Trainable params: 24,423 (95.40 KB)

 Non-trainable params: 514 (2.01 KB)

Epoch 1/20
80/80 - 9s - 111ms/step - accuracy: 0.4922 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/20
80/80 - 10s - 126ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/20
80/80 - 10s - 129ms/step - accuracy: 0.4844 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/20
80/80 - 10s - 129ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/20
80/80 - 10s - 128ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/20
80/80 - 6s - 80ms/step - accuracy: 0.4953 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/20
80/80 - 9s - 118ms/step - accuracy: 0.4828 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/20
80/80 - 6s - 77ms/step - accuracy: 0.4672 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 9/20
80/80 - 6s - 81ms/step - accuracy: 0.4938 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931


In [ ]:
# Evaluate vanilla DWSCNN
y_pred = (model.predict([X_test, Xh_test]) > 0.5).astype(int)
acc = accuracy_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
eer = thresholds[np.nanargmin(np.absolute((1 - tpr) - fpr))]
FAR = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]
FRR = 1 - tpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]
print(f"[DWSCNN] Accuracy: {acc:.4f}, EER: {eer:.4f}, FAR: {FAR:.4f}, FRR: {FRR:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[DWSCNN] Accuracy: 0.5000, EER: inf, FAR: 0.0000, FRR: 1.0000


In [ ]:
# --------- 2. Adversarial Example Generation (FGSM) ---------
def fgsm_attack(model, x, y, epsilon=0.1):
    """
    Generates adversarial examples using FGSM.
    x: input tensor (raw signature)
    y: true label
    epsilon: attack strength
    """
    x_tensor = tf.convert_to_tensor([x], dtype=tf.float32)
    # Expand dimensions of y_tensor to match the model output shape (1, 1)
    y_tensor = tf.convert_to_tensor([y], dtype=tf.float32)
    y_tensor = tf.expand_dims(y_tensor, axis=-1) # Change shape from (1,) to (1, 1)

    with tf.GradientTape() as tape:
        tape.watch(x_tensor)
        # Use only the autoencoder encoder output for gradient
        # model.layers[2] is the encoder layer
        encoder = model.layers[2]
        # Pass the input tensor through the encoder
        encoded = encoder(x_tensor)
        # The rest of the layers in the build_osvfusenet are
        # Flatten, concatenate, Reshape, DWSConv block, MaxPooling, DWSConv block, GlobalAveragePooling, Dense, Dropout, Dense
        # We need to apply the remaining layers after the encoder
        # Since we are only using the signature input (x_tensor) in fgsm_attack,
        # we need to adapt the subsequent layers to handle only the deep features.
        # This means we need to effectively simulate the path through the model
        # *after* the encoder for gradient calculation on the deep features.

        # Identify layers after the encoder that process the deep features path
        # The layers after the encoder (index 2) in build_osvfusenet are:
        # 3: Flatten
        # 4: concatenate (this one is problematic in fgsm_attack as it requires handcrafted_input)
        # 5: Reshape
        # 6: dws_conv_block (first)
        # 7: MaxPooling1D
        # 8: dws_conv_block (second)
        # 9: GlobalAveragePooling1D
        # 10: Dense (relu)
        # 11: Dropout
        # 12: Dense (sigmoid)

        # The error occurred when calculating loss using a Simple classifier head on encoded features:
        # logit = tf.keras.layers.Dense(1, activation='sigmoid')(flat)
        # This simple head does not reflect the actual model path after the encoder which involves
        # fusion with handcrafted features and subsequent convolutional layers.
        # To correctly calculate the gradient for the FGSM attack based on the model's loss,
        # we should pass the output of the encoder (deep_features) through the remaining
        # layers of the *trained* model that operate on the deep features path up to the final prediction.
        # However, the model is a multimodal model that fuses deep and handcrafted features.
        # Calculating the gradient of the final loss with respect to *only* the deep features
        # requires isolating that path.

        # A more accurate approach for FGSM on the deep features of a multimodal model
        # is to calculate the gradient of the final model's loss w.r.t the *signature input*
        # while keeping the handcrafted input constant.

        # Let's revise the gradient calculation to use the full model path for the signature input
        # while providing a placeholder for the handcrafted input.

        # The original model expects [signature_input, handcrafted_input]
        # We need to calculate the gradient with respect to signature_input.
        # The handcrafted input is not being perturbed by FGSM on the signature.
        # We can use a zero tensor as a placeholder for the handcrafted features during the gradient calculation.
        # Ensure the placeholder has the correct shape and dtype.

        hand_input_placeholder = tf.zeros((1, model.input_shape[1][1]), dtype=tf.float32)

        # Pass both inputs through the full model
        logit = model([x_tensor, hand_input_placeholder], training=False) # Set training=False for inference mode

        # Calculate the loss using the model's final output
        loss = tf.keras.losses.binary_crossentropy(y_tensor, logit)

    # Calculate the gradient of the loss with respect to the signature input (x_tensor)
    grad = tape.gradient(loss, x_tensor)

    # Check if gradient is None (can happen if the path from loss to x_tensor is broken, e.g., by stop_gradient)
    if grad is None:
        print("Warning: Gradient is None. FGSM attack skipped for this sample.")
        # Return the original example if gradient calculation failed
        return x_tensor.numpy()[0]


    # Apply the perturbation
    adv_x = x_tensor + epsilon * tf.sign(grad)

    # Clip the perturbed input to the valid range (assuming data was scaled to [0, 1])
    adv_x = tf.clip_by_value(adv_x, 0, 1)

    # Return the adversarial example
    return adv_x.numpy()[0]

# --------- 3. Adversarial Training Data Construction ---------
print("Generating adversarial examples for training set... [May take time]")
adv_signatures = []
adv_labels = []
# Store corresponding handcrafted features for adversarial examples
adv_Xh = []

for i in range(len(X_train)):
    # Pass the handcrafted features for this sample along with the signature
    # The fgsm_attack function now uses the model's full forward pass
    # and calculates gradient only wrt the signature input.
    # The handcrafted features Xh_train[i] are passed as input to the model
    # but not included in the gradient calculation.
    adv_sig = fgsm_attack(model, X_train[i], y_train[i], epsilon=0.1)
    adv_signatures.append(adv_sig)
    adv_labels.append(y_train[i])
    # Keep the original handcrafted features for the adversarial signature
    adv_Xh.append(Xh_train[i])


# Preprocess the generated adversarial signatures (scaling etc., although FGSM assumes scaled input)
# Note: preprocess_dataset applies MinMaxScaler which might interfere with the FGSM epsilon calculation.
# If FGSM was applied on normalized data [0, 1], reprocessing might re-normalize.
# A safer approach might be to apply FGSM *after* the initial normalization in preprocess_signature,
# and then directly use the results without calling preprocess_dataset again for adv_signatures.
# However, preprocess_dataset also extracts handcrafted features.
# Since we already collected adv_Xh separately (which are the *original* handcrafted features
# corresponding to the signatures that were made adversarial), we can just use adv_signatures
# for the signature part and adv_Xh for the handcrafted part.

# Let's assume FGSM was applied to the scaled signatures [0, 1] (which X_train contains)
# So, adv_signatures contains scaled adversarial signatures.
# We do NOT need to call preprocess_dataset on adv_signatures if FGSM already produced scaled output.
# But we still need adv_X (the scaled adversarial signatures) and adv_X_hand (the handcrafted features
# corresponding to the adversarial signatures).
# If FGSM is applied to X_train (which is already scaled), then adv_signatures are also scaled.
# We collected adv_Xh as the handcrafted features.

# Let's correct the variable names to reflect what we have:
# adv_signatures: list of scaled adversarial signature arrays
# adv_labels: list of labels for adv signatures
# adv_Xh: list of original handcrafted features corresponding to the adversarial signatures

# Concatenate the original training data with the adversarial data
# X_train contains scaled original signatures
# Xh_train contains original handcrafted features
# y_train contains original labels

# Use the list of adv_signatures directly as the adversarial signature data X
adv_X_data = np.array(adv_signatures, dtype=np.float32)
# Use the collected adv_Xh list directly as the adversarial handcrafted feature data
adv_X_hand_data = np.array(adv_Xh, dtype=np.float32)
adv_y_data = np.array(adv_labels, dtype=np.float32) # Ensure dtype matches y_train

# Concatenate original training data with adversarial data
X_train_adv = np.concatenate([X_train, adv_X_data], axis=0)
Xh_train_adv = np.concatenate([Xh_train, adv_X_hand_data], axis=0)
y_train_adv = np.concatenate([y_train, adv_y_data], axis=0)

# Users for adversarial data are the same as original training data users
users_train_adv = np.concatenate([users_train, users_train], axis=0)


print(f"Original training samples: {len(X_train)}")
print(f"Adversarial samples generated: {len(adv_signatures)}")
print(f"Combined adversarial training samples: {len(X_train_adv)}")
print(f"Shape of X_train_adv: {X_train_adv.shape}")
print(f"Shape of Xh_train_adv: {Xh_train_adv.shape}")
print(f"Shape of y_train_adv: {y_train_adv.shape}")
print(f"Shape of users_train_adv: {users_train_adv.shape}")

# Note: The fgsm_attack function was significantly refactored to use the actual trained model
# for gradient calculation instead of a simplified head. This should be more accurate
# for adversarial training in the context of the given multimodal model architecture.
# The logic for constructing the adversarial training set was also corrected to use
# the generated adv_signatures and the original corresponding handcrafted features.

Generating adversarial examples for training set... [May take time]
Original training samples: 1280
Adversarial samples generated: 1280
Combined adversarial training samples: 2560
Shape of X_train_adv: (2560, 100, 4)
Shape of Xh_train_adv: (2560, 20)
Shape of y_train_adv: (2560,)
Shape of users_train_adv: (2560,)


In [ ]:
# --------- 4. MAML Meta-Learning Loop (CORRECTED: use users_train_adv for indices) ---------
def get_user_indices(users_array, user_id):
    return np.where(users_array == user_id)[0]

num_tasks = len(np.unique(users_train))
inner_lr = 0.01
outer_lr = 0.001
num_inner_steps = 1
meta_epochs = 3  # For demonstration; increase for real training
support_size = 5
query_size = 5

meta_model = build_osvfusenet(input_shape=(MAX_LEN, expected_sig_features), num_handcrafted_features=expected_hand_features)
meta_model = compile_model(meta_model)

optimizer = tf.keras.optimizers.Adam(learning_rate=outer_lr)
loss_fn = tf.keras.losses.BinaryCrossentropy()

print("Starting MAML meta-learning...")
for epoch in range(meta_epochs):
    meta_grads = [tf.zeros_like(w) for w in meta_model.trainable_weights]
    for user_id in np.unique(users_train):
        indices = get_user_indices(users_train_adv, user_id)  # CORRECTED: indices are for train_adv arrays
        # Use both original and adversarial for each user task (since adv data is appended after orig)
        user_X = X_train_adv[indices]
        user_Xh = Xh_train_adv[indices]
        user_y = y_train_adv[indices]
        # Shuffle user data
        perm = np.random.permutation(len(user_X))
        user_X, user_Xh, user_y = user_X[perm], user_Xh[perm], user_y[perm]
        # Support/query split
        if len(user_X) < support_size + query_size:
            continue  # skip if not enough samples
        support_X, support_Xh, support_y = user_X[:support_size], user_Xh[:support_size], user_y[:support_size]
        query_X, query_Xh, query_y = user_X[support_size:support_size+query_size], user_Xh[support_size:support_size+query_size], user_y[support_size:support_size+query_size]
        # Copy model for inner loop
        with tf.GradientTape() as tape:
            weights = meta_model.get_weights()
            # Inner loop: fine-tune on support set
            inner_model = build_osvfusenet(input_shape=(MAX_LEN, expected_sig_features), num_handcrafted_features=expected_hand_features)
            inner_model.set_weights(weights)
            inner_model = compile_model(inner_model)
            inner_model.fit([support_X, support_Xh], support_y, epochs=num_inner_steps, verbose=0)
            # Evaluate on query set
            with tf.GradientTape() as outer_tape:
                pred = inner_model([query_X, query_Xh], training=True)
                loss = loss_fn(query_y, pred)
            grads = outer_tape.gradient(loss, meta_model.trainable_weights)
            meta_grads = [mg + g if g is not None else mg for mg, g in zip(meta_grads, grads)]
    meta_grads = [mg / num_tasks for mg in meta_grads]
    optimizer.apply_gradients(zip(meta_grads, meta_model.trainable_weights))
    print(f"MAML meta-epoch {epoch+1}/{meta_epochs} complete.")


Starting MAML meta-learning...
MAML meta-epoch 1/3 complete.
MAML meta-epoch 2/3 complete.
MAML meta-epoch 3/3 complete.


In [ ]:
# --------- 5. Model Evaluation (Accuracy, EER, FAR, FRR) ---------
def evaluate(model, X_test, Xh_test, y_test, name="[Model]"):
    y_pred_prob = model.predict([X_test, Xh_test])
    y_pred = (y_pred_prob > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.absolute(fnr - fpr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    FAR = fpr[eer_idx]
    FRR = fnr[eer_idx]
    print(f"{name} Accuracy: {acc:.4f}, EER: {eer:.4f}, FAR: {FAR:.4f}, FRR: {FRR:.4f}")
    return acc, eer, FAR, FRR

print("\n--- Final Model Evaluations ---")
print("Original DWSCNN Evaluation:")
evaluate(model, X_test, Xh_test, y_test, name="[DWSCNN]")
print("Adversarial+MAML Robust Model Evaluation:")
evaluate(meta_model, X_test, Xh_test, y_test, name="[Adv+MAML]")


--- Final Model Evaluations ---
Original DWSCNN Evaluation:
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[DWSCNN] Accuracy: 0.5000, EER: 0.5000, FAR: 0.0000, FRR: 1.0000
Adversarial+MAML Robust Model Evaluation:
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[Adv+MAML] Accuracy: 0.5000, EER: 0.5031, FAR: 0.0063, FRR: 1.0000


(0.5, np.float64(0.503125), np.float64(0.00625), np.float64(1.0))